In [ ]:
import pandas as pd 
import numpy as np
import csv
import os.path
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import sklearn
import seaborn as sns

In [ ]:
#Importing the data from the csv file in the Data folder and turn them into pandas dataframe. Some of them needs to be converted to int datatype to combina all the datasets because some of the dataframe is type object when read'''
champions = pd.read_csv(r'./Data/champs.csv')
matches = pd.read_csv(r'./Data/matches.csv', dtype = np.array(np.dtype('int16')))
stats1 = pd.read_csv(r'./Data/stats1.csv', dtype = np.array(np.dtype('int16')))
stats1_2 = pd.read_csv(r'./Data/stats1-2.csv', dtype = np.array(np.dtype('int16')))
stats1_3 = pd.read_csv(r'./Data/stats1-3.csv', dtype = np.array(np.dtype('int16')))
stats2 = pd.read_csv(r'./Data/stats2.csv', dtype = np.array(np.dtype('int16')))
stats2_2 = pd.read_csv(r'./Data/stats2-2.csv', dtype = np.array(np.dtype('int16')))

#Appending stats1 and stats2 dataset
stats1_2 = pd.concat([stats1_2,stats1_3],sort = False)#stats1_2.concat(stats1_3,, sort = False);
stats1 = pd.concat([stats1,stats1_2],sort = False)#stats1.concat(stats1_2);
stats2 = pd.concat([stats2,stats2_2],sort = False)#stats2.concat(stats2_2);
stats =pd.concat([stats1,stats2],sort = False)# stats1.concat(stats2)

In [ ]:
#Merge particpants with appropriate champion name. test
participants = pd.read_csv(r'./Data/participants.csv', dtype = np.array(np.dtype('int16')))
#Rename the column of the participants dataframe from id to play_id and championid to id for better clarification. inplace so no extra copy is made
participants.rename(columns ={'id': 'player_id','championid':'id'}, inplace = True)
#Merge the participants dataset with the champions dataset based on the championid. This is similar to left join in SQL 
champions['id'] = champions['id'].astype(str).astype(int)
participants['id'] = participants['id'].astype(str).astype(int)
participants = pd.concat([participants.set_index('id'),champions.set_index('id')], axis=1, join='inner').reset_index()
#Delete the column role because I think it's irrelevant for now 

#Rename the stats data set column from id to player_id and inplace 
stats.rename(columns={'id':'player_id'},inplace = True)
#Convert the player_id column to int because it is currently type Object and we cannot manipulate object type column and int type column too easily 
stats['player_id'] = stats['player_id'].astype(int)
participants['player_id'] = participants['player_id'].astype(str).astype(int)
#merge the participants dataset with stats dataset based on the player_id. This makes participants a massive dataset with lots of columns and champions that each player uses
participants = participants.merge(stats, how = 'left', on = 'player_id')
#This is me testing out the gorupby but this will group the champion based on their name and count how many win and lose they have. Win and lose are represented with 0 and 1 
champsFreq = participants.groupby(['name','win'])['win'].count()

#Turn it into a dataframe
champsFreq = pd.DataFrame(champsFreq)
#Add Suffix Count to the second column. When we groupby, it creates columns with same name and it's difficult to select columns so I add a letter to the column with the number of wins and lose for easy access to the dataframe
champsFreq = champsFreq.add_suffix('Count').reset_index()
#Converting the columns from object to int type
champsFreq['win'] = champsFreq['win'].astype(str).astype(int)

In [ ]:
#Get the lose values
loseValues = champsFreq.loc[champsFreq['win'] == 0,'winCount'].reset_index()
loseValues.rename(columns = {'winCount': 'lose'}, inplace = True)
loseValues = loseValues['lose']

#Get the win values
winValues = champsFreq.loc[champsFreq['win'] == 1, 'winCount'].reset_index()
winValues.rename(columns = {'winCount':'win'}, inplace = True)
winValues = winValues['win']

#Get the champion name
championName = champsFreq['name'].drop_duplicates().reset_index()
championName = championName['name']

#Remake the chamions dataframe with win and lose values for each champion to make a clearer dataframe
champions_values = pd.DataFrame([championName, winValues, loseValues]).transpose()

In [ ]:
#get the match id specific columns good for evaluation 
matches.rename(columns = {'id': 'matchid'}, inplace = True)
matches = matches[['matchid','seasonid','duration','version']]
#get the match data for season 8 only
matches['seasonid'] = matches['seasonid'].astype(int)
matches = matches[matches['seasonid'] == 8]
matches['matchid'] = matches['matchid'].astype(int)

In [ ]:
#get participants columns that needed 
participants_match = participants[['id','player_id','matchid','player','name','win']]
#participants_match = participants #we might need more col than this
participants_match['matchid'] = participants_match['matchid'].astype(int)

In [ ]:
#merge on matchid 
new_match = pd.merge(matches, participants_match, on = 'matchid')
new_match['player'] = new_match['player'].astype(int)

In [ ]:
new_match.head()

In [ ]:
#look for inconsistant data in the new_match dataframe
toDel = []
test = [1,2,3,4,5,6,7,8,9,10]
listVal = list(dict.fromkeys(new_match['matchid']))

In [ ]:
for i in listVal:
    listPlayer = list(new_match['player'].loc[new_match['matchid'] == i])
    #check if the list follows the same order of ten players
    if listPlayer == test:
        continue
    else:
        toDel.append(i)

In [ ]:
#Remove all bad rows
new_match = new_match[~new_match.matchid.isin(toDel)]

In [ ]:
#sort the champions_values based on win rate
testPlt = champions_values.sort_values(by = ['win'], ascending = [False])

In [ ]:
#Get the top 5 with the highest win rate
testChamp = testPlt[0:5]

In [ ]:
#plotting it
labels = list(testChamp['name'])
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, testChamp['win'], width, label='Win') #setting the bar width for win rate
rects2 = ax.bar(x + width/2, testChamp['lose'], width, label='Lose') #setting the bar width for lose rate

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Outcomes')
ax.set_title('Game Win/Lose Rate of a Champion')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

In [ ]:
#Remove all bad rows with inconsistent data
stats_for_model = participants[~participants.matchid.isin(toDel)]
stats_for_model

In [ ]:
#Select only the columns that we would like to use
#stats_for_model = stats_for_model[['player_id','matchid','name','win','kills','deaths','assists','totdmgtochamp','totminionskilled']]

In [ ]:
listColumns = list(stats_for_model.columns)
#Drop all NaN or empty value
stats_for_model = stats_for_model.dropna()

In [ ]:
#Convert all column other than name to int
for i in listColumns:
    if i != 'name':
        stats_for_model[i] = stats_for_model[i].astype(int)
    

In [ ]:
#merge with match dataset based on matchid
stats_for_model = stats_for_model.merge(matches, how = 'right', on = 'matchid')

In [ ]:
del stats_for_model['version']
del stats_for_model['duration']

In [ ]:
#This is me testing out the gorupby but this will group the champion based on their name and count how many win and lose they have. Win and lose are represented with 0 and 1 
champStats = stats_for_model.groupby(['name','win'])['win'].count()

#Turn it into a dataframe
champStats = pd.DataFrame(champStats)
#Add Suffix Count to the second column. When we groupby, it creates columns with same name and it's difficult to select columns so I add a letter to the column with the number of wins and lose for easy access to the dataframe
champStats = champStats.add_suffix('Count').reset_index()
#Converting the columns from object to int type
champStats['win'] = champStats['win'].astype(str).astype(int)

#Get the lose values
stats_loseValues = champStats.loc[champStats['win'] == 0,'winCount'].reset_index()
stats_loseValues.rename(columns = {'winCount': 'lose'}, inplace = True)
stats_loseValues = stats_loseValues['lose']

#Get the win values
stats_winValues = champStats.loc[champsFreq['win'] == 1, 'winCount'].reset_index()
stats_winValues.rename(columns = {'winCount':'win'}, inplace = True)
stats_winValues = stats_winValues['win']

#Get the champion name
stats_champName = champStats['name'].drop_duplicates().reset_index()
stats_champName = stats_champName['name']

#Remake the chamions dataframe with win and lose values for each champion to make a clearer dataframe
champ_stats = pd.DataFrame([stats_champName, stats_winValues, stats_loseValues]).transpose()


In [ ]:
#get average kill, deaths, assists, ... for each champion
listChamps = list(stats_champName)
listColumn = ['kills','deaths','assists','totdmgtochamp','totminionskilled']
for colName in listColumn:
    listVal = []
    for champ in listChamps:
        value = stats_for_model[colName].loc[stats_for_model['name'] == champ].mean()
        listVal.append(value)
    champ_stats['Ave ' + colName] = pd.DataFrame(listVal)

In [ ]:
champ_stats